## 텍스트 파일로 변환하는 로직
- asposecells 라이브러리 사용

In [9]:
''' 텍스트파일로 변환하는 로직
import  jpype     
import  asposecells     
from asposecells.api import Workbook
# notebooks → backend/data 로 접근
csv_path = "../backend/data/BookLoan_10years_data.csv"
txt_path = "../backend/data/BookLoan_10years_data.txt"

workbook = Workbook(csv_path)
workbook.save(txt_path)
jpype.shutdownJVM() 
'''

## 저장한 csv파일 로드 및 종류를 분류하여 대출횟수 많은순으로 정렬
1. 도서 종류에 따른 분류 
2. 1학년 1학기 학생 필터링
3. 도서별 대출 횟수 집계
4. 대출 횟수별 정렬

In [35]:
import pandas as pd
from datetime import datetime

# CSV 불러오기
df = pd.read_csv("../backend/data/BookLoan_10years_data.csv", encoding="utf-8")

# 날짜 변환
df["대출일자"] = pd.to_datetime(df["대출일자"])

# 입학년도 추론 (학번 앞 2자리)
df["입학년도"] = 2000 + df["학번"].astype(str).str[:2].astype(int)

# 현재 학년 (최대 4학년)
df["재학년차"] = datetime.now().year - df["입학년도"]
df["학년"] = df["재학년차"].apply(lambda x: x if x <= 4 else 4)

# 학기 계산 (3~9월 = 1학기, 10~2월 = 2학기)
df["월"] = df["대출일자"].dt.month
df["학기"] = df["월"].apply(lambda m: 1 if 3 <= m <= 9 else 2)

# 청구기호 분류 매핑
def classify_subject(code):
    try:
        num = int(str(code).split('.')[0])  # 앞자리 정수만 추출
    except:
        return "기타"

    if 0 <= num <= 99: return "총류"
    elif 100 <= num <= 199: return "철학"
    elif 200 <= num <= 299: return "종교"
    elif 300 <= num <= 399: return "사회과학"
    elif 400 <= num <= 499: return "자연과학"
    elif 500 <= num <= 599: return "기술과학"
    elif 600 <= num <= 699: return "예술"
    elif 700 <= num <= 799: return "언어"
    elif 800 <= num <= 899: return "문학"
    elif 900 <= num <= 999: return "역사"
    else: return "기타"

df["분류"] = df["청구기호"].apply(classify_subject)

# 분야 리스트
subjects = ["총류", "철학", "종교", "사회과학", "자연과학", "기술과학", "예술", "언어", "문학", "역사", "기타"]

# 추천 집계 함수
def recommend_books(year, semester, category, top_n=5):
    subset = df[(df["학년"] == year) & (df["학기"] == semester) & (df["분류"] == category)]

    if subset.empty:
        return pd.DataFrame()  # 빈 DF 반환

    book_counts = (
        subset.groupby("서명")
        .agg(
            대출횟수=("서명", "count"),
            대출학생수=("학번", "nunique")
        )
        .reset_index()
        .sort_values(by="대출학생수", ascending=False)
    )
    return book_counts.head(top_n)

# 전체 결과 저장
results = []

for grade in range(1, 5):       # 1~4학년
    for semester in [1, 2]:     # 1학기, 2학기
        for subject in subjects:
            result = recommend_books(grade, semester, subject)
            if not result.empty:
                result["학년"] = grade
                result["학기"] = semester
                result["분야"] = subject
                results.append(result)

# 리스트 → 하나의 DataFrame으로 합치기
final_df = pd.concat(results, ignore_index=True)

# CSV로 저장
final_df.to_csv("../backend/data/recommend_all.csv", index=False, encoding="utf-8-sig")
print("✅ 전체 결과가 recommend_all.csv 파일로 저장되었습니다!")

✅ 전체 결과가 recommend_all.csv 파일로 저장되었습니다!
